# **Project 4: Advanced Lane Finding** 

# **REFERENCES**
https://github.com/udacity/CarND-Camera-Calibration/blob/master/camera_calibration.ipynb

https://carnd-forums.udacity.com/questions/38543970/p4-birds-eye-transformation-after-masking-throws-error-assertion-failed-ifunc-0-in-remap-file

https://chatbotslife.com/robust-lane-finding-using-advanced-computer-vision-techniques-46875bb3c8aa#.uoh0q8oc2

https://carnd-forums.udacity.com/questions/38535979/p4-are-suggested-reference-points-good-enough

https://carnd-forums.udacity.com/questions/29494501/birds-eye-view-transform

# **LOAD PACKAGES**
Load necessary packages

In [ ]:
#Import Packages
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import pickle
%matplotlib inline
from matplotlib.path import Path
import matplotlib.patches as patches
from scipy import ndimage as ndi

# **LOAD IMAGES**
Load the calibration and test images and check to make sure they loaded.

In [ ]:
# Load Calibration Images
images = glob.glob('camera_cal/calibration*.jpg')
# Load Test Images
testimages = glob.glob('test_images/*.jpg')

# Test to make sure that the camera calibration images are loading properly
testimg   = cv2.imread(images[2],3)
testimg   = cv2.cvtColor(testimg,cv2.COLOR_BGR2RGB)
plt.figure(0)
plt.imshow(testimg)
# Test to make sure that the test images are loading properly
testimg   = cv2.imread(testimages[2],3)
testimg   = cv2.cvtColor(testimg,cv2.COLOR_BGR2RGB)
plt.figure(1)
plt.imshow(testimg)

# ** CAMERA CALIBRATION **
Do corner detection on each of the camera calibration images and compute the camera calibration matricies.

In [ ]:
# Detect Corners
def CornerDetectTest(img):
    nx = 9 #Number of inside corners in x
    ny = 6 #Number of inside corners in y
    # Convert to Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (nx,ny), None)
    # If found, draw coreners
    if ret == True:
        #Draw and display the corners
        cv2.drawChessboardCorners(img,(nx,ny),corners, ret)   
        plt.imshow(img)
# Test to make sure corners are detected
for idx, fname in enumerate(images):
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(idx)
    CornerDetectTest(img)

In [ ]:
# Compute Camera Calibration Parameters
# prepare object points
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for idx, fname in enumerate(images):
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_size = (img.shape[1], img.shape[0])

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6), None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        cv2.drawChessboardCorners(img, (9,6), corners, ret)
        # Save calibrated images to disk
        write_name = 'camera_cal/corners_found'+str(idx)+'.jpg'
        cv2.imwrite(write_name, img)

# Do camera calibration given object points and image points
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)

# Save the camera calibration
dist_pickle = {}
dist_pickle["mtx"]  = mtx
dist_pickle["dist"] = dist
pickle.dump( dist_pickle, open( "output_images/camera_cal.p", "wb" ) )

In [ ]:
# Function to un-distort images
def undistort_imgs(img,outputimg='') :
    # Load Camera Calibration Coefficients
    PickleCal = pickle.load( open( "output_images/camera_cal.p", "rb" ) )
    mtx  = PickleCal["mtx"]
    dist = PickleCal["dist"]
    # Test undistortion on an image
    img_size = (img.shape[1], img.shape[0])
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    if not(outputimg == ''):
        cv2.imwrite(outputimg,dst)
    return dst
# Test to make sure camera calibration worked on all camera calibration images
for idx, fname in enumerate(images):
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(idx)
    write_cal_img = 'output_images/calibration'+str(idx)+'_calibrated.jpg'
    dst = undistort_imgs(img,write_cal_img)    
    # Visualize undistortion
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    ax1.imshow(img)
    ax1.set_title('Original', fontsize=30)
    ax2.imshow(dst)
    ax2.set_title('Undistorted', fontsize=30)

In [ ]:
# Test to make sure camera calibration worked on all test images
for idx, fname in enumerate(testimages):
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(idx)
    write_tst_img = 'output_images/test_images_'+str(idx)+'_calibrated.jpg'
    dst = undistort_imgs(img,write_tst_img)    
    # Visualize undistortion
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    ax1.imshow(img)
    ax1.set_title('Original', fontsize=30)
    ax2.imshow(dst)
    ax2.set_title('Undistorted', fontsize=30)

# ** PERSPECTIVE TRANSFORM **
Compute the birds eye perspective transform on the images

In [ ]:
def BirdsEyeTransform(img):   
    h           = img.shape[0]
    w           = img.shape[1]
    img_size    = (w, h)
    src         = np.float32([[583, 460], [203, 720], [1127, 720], [705, 460]])
    dst         = np.float32([[320, 0], [320, 720], [960,720], [960, 0]])
    M           = cv2.getPerspectiveTransform(src, dst)
    img_size    = (img.shape[1], img.shape[0])
    binary_warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    M_inv       = np.linalg.inv(M)
    return binary_warped, M_inv
# Test that BirdsEyeTransform works
for idx, fname in enumerate(testimages):
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(idx)
    write_tst_img = ''
    img = undistort_imgs(img,write_tst_img)
    bdeyeimg,M_inv = BirdsEyeTransform(img)
    # Take points for plotting source and destination warp
    src       = np.float32([[583, 460], [203, 720], [1127, 720], [705, 460]])
    dst       = np.float32([[320, 0], [320, 720], [960,720], [960, 0]])
    h, w, c     = img.shape
    img_size    = (w, h)
    # Visualize BirdsEyeTransform
    plt.figure(idx)
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    ax1.imshow(img)
    
    # Draw the test image and the polygon
    verts = [src[0], src[1], src[2], src[3], src[0]]
    codes = [Path.MOVETO,Path.LINETO,Path.LINETO,Path.LINETO,Path.CLOSEPOLY]
    path  = Path(verts, codes)
    patch = patches.PathPatch(path, color='red', fill=False, lw=2)
    ax1.add_patch(patch)
    ax1.set_title('Original', fontsize=30)
    ax2.imshow(bdeyeimg)
    verts = [dst[0], dst[1], dst[2], dst[3], dst[0]]
    codes = [Path.MOVETO,Path.LINETO,Path.LINETO,Path.LINETO,Path.CLOSEPOLY]
    path  = Path(verts, codes)
    patch = patches.PathPatch(path, color='red', fill=False, lw=2)
    ax2.add_patch(patch)
    ax2.set_title('Birds Eye', fontsize=30)

# ** COLOR THRESHOLDING TESTS **
Perform color thresholding to pick out the white and yellow lanes in the test images

In [ ]:
#Combined Color Thresholding: Identify White and Yellow Pixels in the image
def ColorThresh(img):
    hsv        = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    hls        = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    maskyellow = cv2.inRange(hsv, np.array([ 20, 0, 0]), np.array([ 30, 255, 255]))
    maskwhite  = cv2.inRange(hls, np.array([ 0, 200, 0]), np.array([ 255, 255, 255]))
    mask       = cv2.bitwise_or(maskyellow,maskwhite)
    out        = cv2.bitwise_and(img,img, mask=mask)
    return out     

for idx, fname in enumerate(testimages):
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(idx)
    write_tst_img = ''
    img, undistort_imgs(img,write_tst_img)
    
    #thresholded = ColorThresh(bdeyeimg)
    r = img[:,:,0]
    g = img[:,:,1]
    b = img[:,:,2]
    color_binary = np.zeros_like(r)
    yellow = [[215, 255], [140, 255], [  0, 160]]
    white  = [[225, 255], [225, 255], [225, 255]]
    color_binary[((( r > 215) & (r <  255)) &(( g > 140) & (g <  255)) &(( b >   0) & (b <  160)))
                 |(((r > 225) & (r <  255)) &(( g > 225) & (g <  255)) &(( b > 225) & (b <  255)))] = 1
    bdeyeimg,M_inv = BirdsEyeTransform(color_binary)
    # Visualize Thresholded Images
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    ax1.imshow(img)
    ax1.set_title('Original', fontsize=30)
    ax2.imshow(bdeyeimg, cmap='gray')
    ax2.set_title('Color Thresholded', fontsize=30)

# ** COLOR SPACES TESTING**
Visualize the different channels of HSV, HSL and RGB to determine which are best to use for gradient thresholding

In [ ]:
# HSV Channels Plotting
for idx, fname in enumerate(testimages):
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    plt.figure(idx)
    img_hsv = undistort_imgs(img_hsv)
    bdeyeimg,M_inv = BirdsEyeTransform(img_hsv)
    h_chan = bdeyeimg[:,:,0]
    s_chan = bdeyeimg[:,:,1]
    v_chan = bdeyeimg[:,:,2]
    # Visualize Thresholded Images
    f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(20,10))
    ax1.imshow(img)
    ax1.set_title('Original', fontsize=30)
    ax2.imshow(h_chan, cmap='gray')
    ax2.set_title('HSV H Channel', fontsize=30)
    ax3.imshow(s_chan, cmap='gray')
    ax3.set_title('HSV S Channel', fontsize=30)
    ax4.imshow(v_chan, cmap='gray')
    ax4.set_title('HSV V Channel', fontsize=30)

In [ ]:
# RGB Channels Plotting
for idx, fname in enumerate(testimages):
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(idx)
    img = undistort_imgs(img)
    bdeyeimg,M_inv = BirdsEyeTransform(img)
    r_chan = bdeyeimg[:,:,0]
    g_chan = bdeyeimg[:,:,1]
    b_chan = bdeyeimg[:,:,2]
    # Visualize Thresholded Images
    f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(20,10))
    ax1.imshow(img)
    ax1.set_title('Original Image', fontsize=30)
    ax2.imshow(r_chan, cmap='gray')
    ax2.set_title('RGB R Channel', fontsize=30)
    ax3.imshow(g_chan, cmap='gray')
    ax3.set_title('RGB G Channel', fontsize=30)
    ax4.imshow(b_chan, cmap='gray')
    ax4.set_title('RGB B Channel', fontsize=30)

In [ ]:
# HLS Channels Plotting
for idx, fname in enumerate(testimages):
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    plt.figure(idx)
    img_hls = undistort_imgs(img_hls)
    bdeyeimg,M_inv = BirdsEyeTransform(img_hls)
    h_chan = bdeyeimg[:,:,0]
    l_chan = bdeyeimg[:,:,1]
    s_chan = bdeyeimg[:,:,2]
    # Visualize Thresholded Images
    f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(20,10))
    ax1.imshow(img)
    ax1.set_title('Original Image', fontsize=30)
    ax2.imshow(h_chan, cmap='gray')
    ax2.set_title('HLS H Channel', fontsize=30)
    ax3.imshow(l_chan, cmap='gray')
    ax3.set_title('HLS L Channel', fontsize=30)
    ax4.imshow(s_chan, cmap='gray')
    ax4.set_title('HLS S Channel', fontsize=30)

# ** GRADIENT COMBINED THRESHOLDING ***
Test different kinds of Sobel gradient thresholding, pick one that works best with color thresholding 

In [ ]:
def sobel_absx(img,sobel_kernel,threshx_min=20,threshx_max=80):
    # SOBEL: Directional X 
    img_dx     = cv2.Sobel(img,cv2.CV_64F, 1, 0)
    img_absx   = np.absolute(img_dx)
    img_sobelx = np.uint8(255*img_absx/np.max(img_absx))
    sobel_dirx = np.zeros_like(img_sobelx)
    sobel_dirx[(img_sobelx >= threshx_min) & (img_sobelx <= threshx_max)] = 1
    return sobel_dirx
def sobel_absy(img,sobel_kernel,threshy_min=80,threshy_max=225):
    # SOBEL: Directional Y 
    img_dy     = cv2.Sobel(img,cv2.CV_64F, 0, 1)
    img_absy   = np.absolute(img_dy)
    img_sobely = np.uint8(255*img_absy/np.max(img_absy))
    sobel_diry = np.zeros_like(img_sobely)
    sobel_diry[(img_sobely >= threshy_min) & (img_sobely <= threshy_max)] = 1
    return sobel_diry
def sobel_mag(img,sobel_kernel,threshmag_min =  20,threshmag_max = 120):  
    # SOBEL: Magnitude
    sobelmx = cv2.Sobel(img,cv2.CV_64F, 1, 0)
    sobelmy = cv2.Sobel(img,cv2.CV_64F, 0, 1)
    mag_sobel    = np.sqrt(sobelmx**2 + sobelmx**2)
    scaled_sobel = np.uint8(mag_sobel*255/np.max(mag_sobel))
    sobel_mag    = np.zeros_like(scaled_sobel)
    sobel_mag[(scaled_sobel>=threshmag_min) & (scaled_sobel<=threshmag_max) ]=1
    return sobel_mag
def sobel_dir(img,sobel_kernel,threshd_min= 0.5,threshd_max= 1.2):
    # SOBEL: Calculate gradient direction
    sobeldx    = cv2.Sobel(img,cv2.CV_64F,1,0, ksize=sobel_kernel)
    sobeldy    = cv2.Sobel(img,cv2.CV_64F,0,1, ksize=sobel_kernel)
    abs_sobelx = np.absolute(sobeldx)
    abs_sobely = np.absolute(sobeldy)
    dir_grad   = np.arctan2(abs_sobely, abs_sobelx)
    sobel_dir    = np.zeros_like(dir_grad)
    sobel_dir[(dir_grad>=threshd_min) & (dir_grad<=threshd_max)] = 1
    return sobel_dir
def combined_color(img):
    r = img[:,:,0]
    g = img[:,:,1]
    b = img[:,:,2]
    color_binary = np.zeros_like(b)
    yellow = [[215, 255], [140, 255], [  0, 160]]
    white  = [[225, 255], [225, 255], [225, 255]]
    color_binary[((( r > 215) & (r <  255)) &(( g > 140) & (g <  255)) &(( b >   0) & (b <  160)))
                 |(((r > 225) & (r <  255)) &(( g > 225) & (g <  255)) &(( b > 225) & (b <  255)))] = 1
    return color_binary
    
for idx, fname in enumerate(testimages):
    img       = cv2.imread(fname)
    img       = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_hls   = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    img_gs    = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    hls       = undistort_imgs(img_hls)
    gs        = undistort_imgs(img_gs)
    rgb       = undistort_imgs(img)
    h_channel = hls[:,:,0]
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    
    color_binary = combined_color(rgb);
    
    #sobel_chan = gs
    sobel_chan = s_channel
    
    bin_out_absx = sobel_absx(sobel_chan,21,20,100)
    bin_out_absy = sobel_absy(sobel_chan,3)
    bin_out_mag  = sobel_mag(sobel_chan,3)
    bin_out_dir  = sobel_dir(sobel_chan,3,120,225)
    combo_test = np.zeros_like(bin_out_absx)
    combo_test[((bin_out_absx == 1) & (color_binary == 1))]= 1
    bdeyeimg_absx,M_inv = BirdsEyeTransform(bin_out_absx)
    bdeyeimg_absy,M_inv = BirdsEyeTransform(bin_out_absy)
    bdeyeimg_mag,M_inv  = BirdsEyeTransform(bin_out_mag)
    bdeyeimg_dir,M_inv  = BirdsEyeTransform(bin_out_dir)
    bdeyeimg_clr,M_inv  = BirdsEyeTransform(combo_test)
    
    # Visualize Thresholded Images
    plt.figure(idx)
    f, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(1, 5, figsize=(20,10))
    ax1.imshow(img)
    ax1.set_title('Original', fontsize=30)
    ax2.imshow(bdeyeimg_absx, cmap='gray')
    ax2.set_title('Sobel ABSx', fontsize=30)
    ax3.imshow(bdeyeimg_absy, cmap='gray')
    ax3.set_title('Sobel ABSy', fontsize=30)
    ax4.imshow(bdeyeimg_mag, cmap='gray')
    ax4.set_title('Sobel Mag', fontsize=30)
    ax5.imshow(bdeyeimg_clr, cmap='gray')
    ax5.set_title('Clr + ABSx', fontsize=30)

# ** COMBINED COLOR & GRADIENT THRESHOLDING **
Test combined color and gradient thresholding to form binary images

In [ ]:
def pipeline_binary_img_test(img_rgb):
    img_hls   = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HLS)
    img_gs    = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    hls       = undistort_imgs(img_hls)
    gs        = undistort_imgs(img_gs)
    rgb       = undistort_imgs(img_rgb)
    h_channel = hls[:,:,0]
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    
    color_binary = combined_color(rgb);
    
    sobel_chan = gs
    #sobel_chan = s_channel
    
    bin_out_absx = sobel_absx(sobel_chan,3,20,100)
    combo_test = np.zeros_like(bin_out_absx)
    combo_test[((bin_out_absx == 1) | (color_binary == 1))]= 1
    
    return combo_test

for idx, fname in enumerate(testimages):
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(idx)
    thresholded    = pipeline_binary_img_test(img)
    bdeyeimg,M_inv = BirdsEyeTransform(thresholded)
    # Visualize Thresholded Images
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    ax1.imshow(img)
    ax1.set_title('Original', fontsize=30)
    ax2.imshow(bdeyeimg, cmap='gray')
    ax2.set_title('Thresholded', fontsize=30)

In [ ]:
# In all of the frames the lanes appear within the x range of 200 to 1200, we want to remove extra points
# outside this range that are giving false measurements, this includes things like the concrete road dividers
# on the left side of the image.

def region_of_interest(img, vertices):
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def filter_outliers(thresholded, filtersize):

    filtered = np.zeros_like(thresholded)
    
    label_objects, nb_labels = ndi.label(thresholded)
    sizes = np.bincount(label_objects.ravel())

    mask_sizes = sizes > filtersize
    mask_sizes[0] = 0
    thresholded = mask_sizes[label_objects]
    
    whites = thresholded > 0
    
    filtered[whites] = 1
    
    return filtered

def mask_bdseye(bdeyeimg):
    imshape         = bdeyeimg.shape
    x_dim           = imshape[1]
    y_dim           = imshape[0]
    vertices        = np.array([ [(200,0), (200,720), (1200,720), (1200,0)]], dtype=np.int32)
    masked_img      = region_of_interest(bdeyeimg, vertices)
    return masked_img

# Test to make sure masking worked
for idx, fname in enumerate(testimages):
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(idx)
    thresholded    = pipeline_binary_img_test(img)
    bdeyeimg,M_inv = BirdsEyeTransform(thresholded)  
    bdeyeimg       = filter_outliers(bdeyeimg,30)
    masked_img     = mask_bdseye(bdeyeimg)
    # Visualize undistortion
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    plotvertices    = np.float32([[200,0], [200,720], [1200,720], [1200,0]])
    verts = [plotvertices[0], plotvertices[1], plotvertices[2], plotvertices[3], plotvertices[0]]
    codes = [Path.MOVETO,Path.LINETO,Path.LINETO,Path.LINETO,Path.CLOSEPOLY]
    path  = Path(verts, codes)
    patch = patches.PathPatch(path, color='red', fill=False, lw=2)
    ax1.imshow(bdeyeimg)
    ax1.add_patch(patch)
    ax1.set_title('Original', fontsize=30)
    ax2.imshow(masked_img)
    ax2.set_title('Masked', fontsize=30)

# ** IMAGE PREPROCESSING TEST **
Test that the combined image processing is working properly

In [ ]:
def preprocessimage(img):
    # Apply Distortion Correction
    undistortedimg = undistort_imgs(img)
    # Apply Binary Thresholding
    thresholded     = pipeline_binary_img_test(img)
    # Apply Perspective Transform
    bdeyeimg,M_inv = BirdsEyeTransform(thresholded)
    bdeyeimg       = filter_outliers(bdeyeimg,30)
    bdeyeimg       = mask_bdseye(bdeyeimg)
    yield bdeyeimg
    yield undistortedimg
    yield M_inv
 
for idx, fname in enumerate(testimages):
    img            = cv2.imread(fname)
    img            = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_proc,undist,M_inv = preprocessimage(img)
    # Visualize Preprocessed images
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    ax1.imshow(undist)
    ax1.set_title('Original Image', fontsize=30)
    ax2.imshow(img_proc, cmap='gray')
    ax2.set_title('Preprocessed Image', fontsize=30)

# ** LANE DETECTION **
Perform lane detection algorithms

In [ ]:
# Draw Histograms to Visualize Detection of Lanes
figure = plt.figure(1,figsize=(18, 20))
for idx, fname in enumerate(testimages):
    img      = cv2.imread(fname)
    img      = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_proc,undist,M_inv = preprocessimage(img)
    # Visualize Histograms Across Preprocessed Images
    figure.add_subplot(4,2,idx+1)
    plt.imshow(img_proc, cmap='gray')
    histogram = np.sum(img_proc[img_proc.shape[0]/2:,:], axis=0)
    plt.plot(histogram,'r')
    plt.ylabel('Y Pixels / Counts')
    plt.xlabel('X Pixels / Pixel Position')

In [ ]:
def lane_find(img):
    # Assuming you have created a warped binary image
    # Take a histogram of the bottom half of the image
    histogram = np.sum(img[img.shape[0]/2:,:], axis=0)
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((img, img, img))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint    = np.int(histogram.shape[0]/2)
    leftx_base  = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    # Choose the number of sliding windows
    nwindows    = 9
    # Set height of windows
    window_height = np.int(img.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero  = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current  = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 110
    # Set minimum number of pixels found to recenter window
    minpix = 100
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds  = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low       = img.shape[0]  - (window+1)*window_height
        win_y_high      = img.shape[0]  - window*window_height
        win_xleft_low   = leftx_current  - margin
        win_xleft_high  = leftx_current  + margin
        win_xright_low  = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds  = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds  = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx  = nonzerox[left_lane_inds]
    lefty  = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit  = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    ploty      = np.linspace(0, img.shape[0]-1, img.shape[0] )
    left_fitx  = left_fit[0]*ploty**2 +  left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    yield out_img
    yield left_fitx
    yield right_fitx
    yield left_fit
    yield right_fit
    yield ploty
    yield left_lane_inds
    yield right_lane_inds
    yield nonzerox
    yield nonzeroy

figure = plt.figure(1,figsize=(18, 20))
for idx, fname in enumerate(testimages):
    img            = cv2.imread(fname)
    img            = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_proc,undist,M_inv = preprocessimage(img)
    out_img,left_fitx,right_fitx,left_fit,right_fit,ploty,left_lane_inds,right_lane_inds,nonzerox,nonzeroy = lane_find(img_proc)
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]]   = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    figure.add_subplot(4,2,idx+1)    
    plt.imshow(out_img, cmap='gray')
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    plt.xlim(0, 1280)
    plt.ylim(720, 0)    

In [ ]:
def nextframe_lanefind(nxt_img,left_fit,right_fit,Debug=False):
    # Assume you now have a new warped binary image from the next frame of video 
    nonzero  = nxt_img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin   = 90
    
    leftLaneInds  = ((nonzerox > ( left_fit[0]*(nonzeroy**2) +  left_fit[1]*nonzeroy +  left_fit[2] - margin)) & (nonzerox < ( left_fit[0]*(nonzeroy**2) +  left_fit[1]*nonzeroy +  left_fit[2] + margin))) 
    rightLaneInds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  

    # Again, extract left and right line pixel positions
    leftx  = nonzerox[ leftLaneInds]
    lefty  = nonzeroy[ leftLaneInds] 
    rightx = nonzerox[rightLaneInds]
    righty = nonzeroy[rightLaneInds]
    
    # Fit a new second order polynomial to each lane line
    left_fit  = np.polyfit( lefty,  leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Generate x and y values for plotting
    ploty      = np.linspace(0, nxt_img.shape[0]-1, nxt_img.shape[0] )
    left_fitx  = left_fit[0]*ploty**2 +  left_fit[1]*ploty +  left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    # PLOTTING STUFF #
    if(Debug):
        # Create an image to draw on and an image to show the selection window
        out_img = np.dstack((nxt_img, nxt_img, nxt_img))*255
        window_img = np.zeros_like(out_img)

        # Color in left and right line pixels
        out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]]   = [255, 0, 0]
        out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

        # Generate a polygon to illustrate the search window area
        # And recast the x and y points into usable format for cv2.fillPoly()
        left_line_window1  = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
        left_line_window2  = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, ploty])))])
        left_line_pts      = np.hstack((left_line_window1, left_line_window2))
        right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
        right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, ploty])))])
        right_line_pts     = np.hstack((right_line_window1, right_line_window2))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
        cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
        result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    else:
        out_img= []
        result = []
    
    # OUTPUTS #
    
    yield out_img
    yield left_fitx
    yield right_fitx
    yield left_fit
    yield right_fit
    yield ploty
    yield result

figure = plt.figure(1,figsize=(18, 20))
for idx, fname in enumerate(testimages):
    img      = cv2.imread(fname)
    img      = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_proc,undist,M_inv = preprocessimage(img)
    out_img,left_fitx,right_fitx,left_fit,right_fit,ploty,left_lane_inds,right_lane_inds,nonzerox,nonzeroy  = lane_find(img_proc)
    out_img,left_fitx,right_fitx,left_fit,right_fit,ploty,result = nextframe_lanefind(img_proc,left_fit,right_fit,True)
    figure.add_subplot(4,2,idx+1)
    plt.imshow(result)
    plt.plot(left_fitx,  ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    plt.xlim(0, 1280)
    plt.ylim(720, 0)

# ** Compute Lane Curvature and Error **

In [ ]:
def ComputeLaneCurvature(fity, leftx, rightx, ploty):
    y_eval = np.max(ploty)
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720  # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    # Fit new polynomials to x,y in world space (meters)
    left_fit_cr  = np.polyfit(ploty*ym_per_pix, leftx* xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, rightx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    left_curverad  = ((1 + (2*left_fit_cr[0] *y_eval*ym_per_pix +  left_fit_cr[1])**2)**1.5) / np.absolute(2* left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    # Now our radius of curvature is in meters
    #print(left_curverad, 'm', right_curverad, 'm')
    # Example values: 632.1 m    626.2 m
    yield left_curverad
    yield right_curverad
    
def ComputeLanePosErr(left, right, width, height):
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720  # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension    
    center = (right[0] + left[0]) / 2
    err    = center - width/2
    posError = err * xm_per_pix
    return posError, err

# ** PROJECT LANE DETECTED TO IMAGE**
Test the projection of the detected lane onto the test images

In [ ]:
def projectlane(warped, M_inv, undist, left_fitx, right_fitx, ploty):
                
    # Create an image to draw the lines on
    warp_zero  = np.zeros_like(undist).astype(np.uint8)

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left  = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts       = np.hstack((pts_left, pts_right))
    
    # Draw the lane onto the warped blank image
    color_warp = cv2.fillPoly(warp_zero, pts.astype(int), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, M_inv, (undist.shape[1], undist.shape[0])) 
    
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
        
    return result

figure = plt.figure(1,figsize=(18, 20))
for idx, fname in enumerate(testimages):
    img            = cv2.imread(fname)
    img            = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    img_proc,undist,M_inv = preprocessimage(img)
    
    out_img,left_fitx,right_fitx,left_fit,right_fit,ploty,left_lane_inds,right_lane_inds,nonzerox,nonzeroy  = lane_find(img_proc)
    out_img,left_fitx,right_fitx,left_fit,right_fit,ploty,result = nextframe_lanefind(img_proc,left_fit,right_fit)
    result  = projectlane(img_proc, M_inv, undist, left_fitx, right_fitx, ploty)
    
    figure.add_subplot(4,2,idx+1)   
    # Compute Lane Curvature and Position Error
    left_curverad,right_curverad = ComputeLaneCurvature(ploty, left_fitx, right_fitx, ploty)
    posError, err                = ComputeLanePosErr(left_fitx, right_fitx, width=img.shape[1], height=img.shape[0])
    # Draw text on the screen (in unwarped image space).  
    # Show position error and left/right curvature values.
    font             = cv2.FONT_HERSHEY_SIMPLEX
    posErrorString   = "Pos Error   = %6.2f m" % (posError)
    cv2.putText(result, posErrorString,(50,50), font, 2, (255,255,255),2)
    leftLaneCurvStr  = "Left Curve  = %6.2f m" % (left_curverad)
    rightLaneCurvStr = "Right Curve = %6.2f m" % (right_curverad)
    cv2.putText(result, leftLaneCurvStr,(50,100), font, 2, (255,255,255),2)
    cv2.putText(result, rightLaneCurvStr,(50,150), font, 2, (255,255,255),2)
    plt.imshow(result)
    plt.xlim(0, 1280)
    plt.ylim(720, 0)

# ** VIDEO PROCESSING PIPELINE **
Test video processing pipeline

In [ ]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        self.detected       = False                            # was the line detected in the last iteration?
        self.left_fit       = None                             #previous frame left lane line fit
        self.right_fit      = None                             #previous frame right lane line fit 
        self.left_fitx      = None                             #previous frame left lane line x fit
        self.right_fitx     = None                             #previous frame right lane line x fit
        self.savedframecnt  = 0

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

def process_video(frame,GenDebugImgs = False):
    
    if(GenDebugImgs):
        cv2.imwrite("frame%i.jpg" % line.savedframecnt, frame)
        line.savedframecnt = line.savedframecnt +1
    
    # Apply Distortion Correction
    undistortedimg = undistort_imgs(frame)
    # Apply Binary Thresholding
    binary_img     = pipeline_binary_img_test(frame)
    # Apply Birds Eye Perspective Transform
    Tfm_img,M_inv  = BirdsEyeTransform(binary_img)
    masked_img     = mask_bdseye(Tfm_img)
    
    # Get the lane from the first frame, update the line class 
    if line.detected == False:
        
        out_img,left_fitx,right_fitx,left_fit,right_fit,ploty,left_lane_inds,right_lane_inds,nonzerox,nonzeroy = lane_find(masked_img)
        line.left_fit   = left_fit
        line.right_fit  = right_fit
        line.left_fitx  = left_fitx
        line.right_fitx = right_fitx
        
        line.detected = True
    # Do a limited search based on the previous lines    
    else:
        out_img,left_fitx,right_fitx,left_fit,right_fit,ploty,result = nextframe_lanefind(masked_img,line.left_fit,line.right_fit)
        
        #If the leading coefficient has changed by a factor of 10 then there is a frame where the lane finding is bad
        #we should weigh the last frames data more than the current frame
        if abs(left_fit[0]) > 10*abs(line.left_fit[0]):
            wt_leftprev  = np.zeros_like(line.left_fitx)
            wt_leftcur   = np.zeros_like(left_fitx)
            wt_rightprev = np.zeros_like(line.right_fitx)
            wt_rightcur  = np.zeros_like(right_fitx)
            
            wt_leftprev  = wt_leftprev  + 0.5
            wt_leftcur   = wt_leftcur   + 0.5
            wt_rightprev = wt_rightprev + 0.5
            wt_rightcur  = wt_rightcur  + 0.5
            
            left_fitx  = np.hstack(( line.left_fitx, left_fitx))
            right_fitx = np.hstack((line.right_fitx,right_fitx))
            
            wt_left  = np.hstack(( wt_leftprev, wt_leftcur))
            wt_right = np.hstack((wt_rightprev,wt_rightcur))
            
            ploty = np.hstack((ploty,ploty))
            
            left_fit  = np.polyfit(ploty, left_fitx,2,w=wt_left)
            right_fit = np.polyfit(ploty,right_fitx,2,w=wt_right)
            
            line.left_fit  = (left_fit  + 24* line.left_fit)/25
            line.right_fit = (right_fit + 24*line.right_fit)/25
            
            left_fitx  =  left_fit[0]*(ploty**2) +  left_fit[1]*ploty +  left_fit[2]
            right_fitx = right_fit[0]*(ploty**2) + right_fit[1]*ploty + right_fit[2]
            
        else:
            line.left_fit  = ( left_fit + 24* line.left_fit)/25
            line.right_fit = (right_fit + 24*line.right_fit)/25
            
            line.left_fitx = left_fitx
            line.right_fitx = right_fitx
    
    # Project the detected lane     
    result  = projectlane(Tfm_img, M_inv, undistortedimg, left_fitx, right_fitx, ploty)
    
    # Compute Lane Curvature and Position Error
    left_curverad,right_curverad = ComputeLaneCurvature(ploty, left_fitx, right_fitx, ploty)
    posError, err                = ComputeLanePosErr(left_fitx, right_fitx, width=img.shape[1], height=img.shape[0])
    # Draw text on the screen (in unwarped image space).  
    # Show position error and left/right curvature values.
    font             = cv2.FONT_HERSHEY_SIMPLEX
    posErrorString   = "Pos Error   = %6.2f m" % (posError)
    cv2.putText(result, posErrorString,(50,50), font, 2, (255,255,255),2)
    leftLaneCurvStr  = "Left Curve  = %6.2f m" % (left_curverad)
    rightLaneCurvStr = "Right Curve = %6.2f m" % (right_curverad)
    cv2.putText(result, leftLaneCurvStr,(50,100), font, 2, (255,255,255),2)
    cv2.putText(result, rightLaneCurvStr,(50,150), font, 2, (255,255,255),2)
    
    return result

output = 'output_videos/project_video.mp4'
vidclip = VideoFileClip("project_video.mp4")#.subclip(20,25)
line = Line()
# Process video
clip = vidclip.fl_image(process_video)
clip.write_videofile(output, audio=False)